In [1]:
import os
import pandas as pd
import math
import numpy as np


def mf(records):
    # 计算FFT和幅度谱
    fft = np.fft.fft(records)
    magnitude = np.abs(fft)

    # 对幅度谱进行排序，并记录每个幅度值对应的频率
    sort_idx = np.argsort(magnitude)
    sorted_mag = magnitude[sort_idx]
    freqs = np.fft.fftfreq(len(records), d=1/2148)[sort_idx]

    # 计算幅度谱的累积和，归一化到总和的一半
    cumsum_mag = np.cumsum(sorted_mag)
    normalized_cumsum_mag = cumsum_mag / np.sum(sorted_mag)
    median_mag = np.interp(0.5, normalized_cumsum_mag, sorted_mag)

    # 找到幅度等于中位数的那个频率
    median_freq = np.interp(median_mag, sorted_mag, freqs)
    return median_freq


def mpf(records):
    # 计算FFT
    fft = np.fft.fft(records)

    # 计算幅度谱和相位谱
    magnitude = np.abs(fft)

    # 计算信号的总功率
    total_power = np.sum(magnitude**2)

    # 计算每个频率的功率贡献
    power_contribution = magnitude**2 / total_power

    # 计算平均功率频率
    mean_power_freq = np.sum(power_contribution * np.fft.fftfreq(len(records))) * np.pi
    return mean_power_freq


# def ssc(records):
#     length = len(records)
#     sum = 0
#
#     for i in range(1, length-1):
#         sum = 0
#         for i in range(1, len(records)-1):
#             temp = (records[i] - records[i-1]) * (records[i] - records[i-1])
#             if temp > 50:
#                 sum = sum + 1
#
#     data = sum
#     return data

def ssc(records):
    length = len(records)
    count = 0

    for i in range(1, length-1):
        temp = (records[i] - records[i-1]) * (records[i] - records[i+1])
        if temp > 2.5e-12:
            count += 1

    return count

def waveLength(records):
    length = len(records)
    sum = 0
    for i in range(length - 1):
        sum = sum + abs(records[i + 1] - records[i])
    data = sum
    return data



def Mav(records):
    length = len(records)
    sum = 0
    count = 0
    for i in range(length):
        count = count + 1
        sum = sum + abs(records[i])

    value = sum / count
    data = value

    return data


def get_rms(records):
    """
    均方根值 反映的是有效值而不是平均值
    """
    return math.sqrt(sum([x ** 2 for x in records]) / len(records))


def sampEn(L: np.array, std: float, m: int = 2, r: float = 0.15):
    """
    计算时间序列的样本熵

    Input:
        L: 时间序列
        std: 原始序列的标准差
        m: 1或2
        r: 阈值

    Output:
        SampEn
    """
    N = len(L)
    B = 0.0
    A = 0.0

    # Split time series and save all templates of length m
    xmi = np.array([L[i:i + m] for i in range(N - m)])
    xmj = np.array([L[i:i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r * std) - 1 for xmii in xmi])
    # Similar for computing A
    m += 1
    xm = np.array([L[i:i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r * std) - 1 for xmi in xm])
    # Return SampEn

    if A > 0 and B > 0:
        return -np.log(A / B)
    else:
        # 当没有匹配时，返回一个高值而不是无穷大，反映高度不可预测性
        return np.max([N, 10])  # 可以根据具体情况选择一个适当的高值



In [2]:
filePath = './data/force'
# filePath = './data_mvc/'
count = 0
featureArr = []
for root, dirs, files in os.walk(filePath):
    for file in files:
        path = os.path.join(root, file)
        print(path)
        labelOne = path.split("/")[-2]
        # print(labelOne)
        data = pd.read_csv(path, header=1)
        # print(data.head())
        length=len(data)
        data = np.array(data)
        data = data[:, 2:8].astype(np.float32)
        col_means = data.mean(axis=0)
        # 从每一列中减去其平均值去除直流偏置
        data = data - col_means
        data = data.T
        # print(data[:5,:])
        print(data.shape)
        # window_size = 2000
        window_size = 50 #滑动窗口大小
        window_num_in_raw=3#每个样本有几个窗口
        raw_length=window_num_in_raw*window_size#每个样本有几条数据
        # window_num=5
        raw_num=int((length)/raw_length)#样本数
        
        
        for j in range(raw_num):#循环样本数
            feature = []
            start_point=j*raw_length#每个样本起始点
            # print(start_point)
            for k in range (window_num_in_raw):#每个raw有多少个窗口
                window_start=start_point+k*window_size#每个窗口起始点
                window_end=start_point+(k+1)*window_size#每个窗口结束点
                for i in range(0, 6):#遍历六个通道
                    temp = data[i][window_start:window_end]
                    f1 = np.std(temp)#计算窗口数据的标准差，反映了数据的变异程度
                    feature.append(f1)
                    feature.append(get_rms(temp))#计算窗口数据的均方根值
                    feature.append(sampEn(temp, std=f1))
                    feature.append(Mav(temp))#平均值
                    feature.append(waveLength(temp))#波长
                    feature.append(mf(temp))
                    feature.append(mpf(temp))
            feature.append(labelOne)
            featureArr.append(feature)

# pd.DataFrame(featureArr).to_csv("./feature/feautre_time_frequency_domain.csv", header=None, index=None, columns=None)
pd.DataFrame(featureArr).to_csv("./feature/force/feautre_nomvc.csv", header=None, index=None, columns=None)


./data/force/b/emg_left_arm_b.csv
(6, 8642)
./data/force/b/emg_left_arm_b_7.csv
(6, 14336)
./data/force/b/emg_left_arm_b_17.csv
(6, 13243)
./data/force/b/emg_left_arm_b_16.csv
(6, 12489)
./data/force/b/emg_left_arm_b_3.csv
(6, 19621)
./data/force/b/emg_left_arm_b_6.csv
(6, 7899)
./data/force/b/emg_left_arm_b_10.csv
(6, 8742)
./data/force/b/emg_left_arm_b_11.csv
(6, 1277)
./data/force/b/emg_left_arm_b_1.csv
(6, 10609)
./data/force/b/emg_left_arm_b_12.csv
(6, 12104)
./data/force/b/emg_left_arm_b_13.csv
(6, 12072)
./data/force/b/emg_left_arm_b_8.csv
(6, 12822)
./data/force/b/emg_left_arm_b_9.csv
(6, 9737)
./data/force/b/emg_left_arm_b_2.csv
(6, 14075)
./data/force/b/emg_left_arm_b_5.csv
(6, 17343)
./data/force/b/emg_left_arm_b_14.csv
(6, 8719)
./data/force/b/emg_left_arm_b_4.csv
(6, 11714)
./data/force/b/emg_left_arm_b_15.csv
(6, 12691)
./data/force/a/emg_left_arm_a_3.csv
(6, 2514)
./data/force/a/emg_left_arm_a_17.csv
(6, 12932)
./data/force/a/emg_left_arm_a_1.csv
(6, 16575)
./data/force/